In [2]:
# # # Requerimientos de librerías:
# !pip install spacy
# !python -m spacy download es_core_news_md

# !git clone https://github.com/JorgeSauri/COMSINT.git 

In [3]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
nlp = spacy.load("es_core_news_md")


In [4]:
basedir = 'datasets/'

df_nutricion = pd.read_csv(basedir + 'nutricion.csv', encoding='latin')
df_nutricion

,Unnamed: 0,nombre,serving_size,kcal,total_fat,saturated_fat,cholesterol,sodium,choline,folate,...,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water
0,0,Maicena,100 g,381,0.1g,NaN,0,9.00 mg,0.4 mg,0.00 mcg,...,0.05 g,0.009 g,0.016 g,0.025 g,0.00 mg,0.0 g,0.09 g,0.00 mg,0.00 mg,8.32 g
1,1,"Nueces, nueces",100 g,691,72g,6.2g,0,0.00 mg,40.5 mg,22.00 mcg,...,71.97 g,6.180 g,40.801 g,21.614 g,0.00 mg,0.0 g,1.49 g,0.00 mg,0.00 mg,3.52 g
2,2,"Berenjena, cruda",100 g,25,0.2g,NaN,0,2.00 mg,6.9 mg,22.00 mcg,...,0.18 g,0.034 g,0.016 g,0.076 g,0.00 mg,0.0 g,0.66 g,0.00 mg,0.00 mg,92.30 g
3,3,"Teff, crudo",100 g,367,2.4g,0.4g,0,12.00 mg,13.1 mg,0,...,2.38 g,0.449 g,0.589 g,1.071 g,0,0,2.37 g,0,0,8.82 g
4,4,"Sherbet, naranja",100 g,144,2g,1.2g,1mg,46.00 mg,7.7 mg,4.00 mcg,...,2.00 g,1.160 g,0.530 g,0.080 g,1.00 mg,0.0 g,0.40 g,0.00 mg,0.00 mg,66.10 g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8784,8784,"Carne de res, crudas, todas las calificaciones...",100 g,125,3.5g,1.4g,62mg,54.00 mg,64.5 mg,4.00 mcg,...,3.50 g,1.353 g,1.554 g,0.244 g,62.00 mg,0.0 g,1.11 g,0.00 mg,0.00 mg,72.51 g
8785,8785,"Cordero, cocinado, separable solo, compuesto d...",100 g,206,8.9g,3.9g,109mg,50.00 mg,0,0.00 mcg,...,8.86 g,3.860 g,3.480 g,0.520 g,109.00 mg,0,1.60 g,0,0,59.95 g
8786,8786,"Cordero, bruto, de grasa y grasa separable, co...",100 g,277,23g,12g,78mg,39.00 mg,0,1.00 mcg,...,22.74 g,11.570 g,8.720 g,0.980 g,78.00 mg,0,0.92 g,0,0,59.80 g
8787,8787,"Carne de res, crudas, todas las calificaciones...",100 g,121,3g,1.1g,60mg,53.00 mg,64.2 mg,4.00 mcg,...,3.04 g,1.086 g,1.266 g,0.233 g,60.00 mg,0.0 g,1.10 g,0.00 mg,0.00 mg,73.43 g


In [5]:
df_profeco = pd.read_csv(basedir + 'lista_precios_profeco_2022.csv', encoding='latin')
df_profeco

,Unnamed: 0,producto,precio_prom_por_gramo
0,0,aceite,0.058843
1,1,acelga,0.077439
2,2,agua con gas,0.045434
3,3,agua sin gas,0.009465
4,4,aguacate,0.051093
...,...,...,...
208,208,vainilla,0.061177
209,209,verdolaga,0.113265
210,210,vinagre,0.018318
211,211,yoghurt,0.051053


In [6]:
def LimpiarString(cadena):
      """
      Limpia una cadena de caracteres extraños, simbolos, stopwords, y unidades de medida

      @cadena: String a limpiar

      Devuelve: La cadena limpia
      """
      result = []
      for c in list(cadena.lower()):
          if (c == ';' or c == '+' or c == '-'):
              c = ','
          else:
              if (not c.isalpha()):
                  c = ' '
          result.append(c)
      result = ''.join(result).split(' ')

      result2 = []
      for e in result:
          if (e != ''):
              result2.append(e)

      result = ''
      for e in result2:
          result += str(e) + ' '

      stopwords = ["el", "para", "con", "en", ",", "contra",
                                "de", "del", "la", "las", "los", "un",
                                "una", "unos", "unas", "o", "ó", "y"]



      # Eliminar las stopwords comunes
      result = ' '.join([word for word in result.split(' ') if word not in stopwords])

      return str(result)


In [28]:

Alimentos = []
Cals = []
Carbs = []
Prots = []
Grasas = []

for n in tqdm(range(len(df_nutricion))):
  filan = df_nutricion.iloc[n]
  alimenton  = LimpiarString(filan['nombre'])
  doc_n = nlp(alimenton)  
  for p in range(len(df_profeco)):
      filap = df_profeco.iloc[p]
      alimentop = LimpiarString(filap['producto'])
      doc_p = nlp(alimentop)      
      sim  = doc_n.similarity(doc_p)
      if (sim >= 0.98):                
        Alimentos.append(alimentop.strip() + ' - ' + alimenton)
        Cals.append(filan['kcal'])
        Carbs.append(filan['carbohydrate'])
        Prots.append(filan['protein'])
        Grasas.append(filan['total_fat'])
        break

servings = ['100 g'] * len(Alimentos)
df_nutricion2 = pd.DataFrame(list(zip(Alimentos, servings, Cals, Prots, Carbs, Grasas)),
                                   columns=['nombre', 'serving_size','kcal', 'protein', 'carbohydrate', 'total_fat']) 

df_nutricion2.to_csv('datasets/nutricion_profeco.csv')
df_nutricion2 

  0%|          | 0/8789 [00:00<?, ?it/s]C:\Users\jsaur\AppData\Local\Temp\ipykernel_10176\1581771585.py:15: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim  = doc_n.similarity(doc_p)
  0%|          | 26/8789 [00:32<3:40:54,  1.51s/it]

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
from recomendaciones_comsint import Recomendador


version = 4

recomendador128 = Recomendador(fuente='recetario_mexicano_small.csv',
                            nutricion = 'nutricion_profeco.csv',
                            canasta='canasta_basica.csv')

recomendador128.CargarModelo(emb_size=128, version=version)

recomendador128.NUM_RECETAS = 2000
recomendador128.EMB_SIZE = 128
ITER = 3
INITIAL_EPOCH = 0
EPOCHS = 50

Histories = []
for iteracion in range(ITER):
  MINU = 1 
  MAXU = 11   
  print('\nITERACIÓN:', iteracion+1)
  print('min unidades:',MINU, ' max unidades:', MAXU)
  print('Entrenando desde epoch', INITIAL_EPOCH)
  print('------------------------------------------\n')
  modelo128, history = recomendador128.EntrenarModelo(df_nutricionales='nutricion_profeco.csv',
                                #df_training='recipes_con_info_nutricional.csv',
                                df_test='recetas_test.csv', 
                                df_val='recetas_val.csv',
                                learning_rate=1e-4,
                                version=version, 
                                initial_epoch = INITIAL_EPOCH,                             
                                epochs=INITIAL_EPOCH + EPOCHS, 
                                kernels=128,                                             
                                min_ingredientes=5, max_ingredientes=11,
                                min_unidades=MINU, max_unidades=MAXU,                               
                                save=True, verbose=True)
  INITIAL_EPOCH = history.epoch[-1]

  Histories.append(history)


In [ ]:
modelo128, history2 = recomendador128.EntrenarModelo(df_nutricionales='nutricion_profeco.csv',
                            df_training='recipes_con_info_nutricional.csv',
                            df_test='recetas_test.csv', 
                            df_val='recetas_val.csv',
                            learning_rate=1e-4,
                            version=version, 
                            initial_epoch = INITIAL_EPOCH,                             
                            epochs=INITIAL_EPOCH + EPOCHS, 
                            kernels=128,                                             
                            min_ingredientes=5, max_ingredientes=11,
                            min_unidades=MINU, max_unidades=MAXU,                               
                            save=True, verbose=True)

Histories.append(history)

In [ ]:
for i in range(len(Histories)):
    history = Histories[i]    
    pd.DataFrame(history.history).plot()
    plt.title('ITERACIÓN ' + str(i))
    plt.show()
    print('LOSS:',history.history['loss'][-1], ' -- MAE:', history.history['mae'][-1], 
          ' -- VAL_LOSS:', history.history['val_loss'][-1], ' -- VAL_MAE:', history.history['val_mae'][-1])
    print('----------------------------------------------------------')